In [ ]:
import os
from PIL import Image

def label_images(image_folder):
    labeled_data = []
    for filename in os.listdir(image_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Add other file types if needed
            image_path = os.path.join(image_folder, filename)
            image = Image.open(image_path)
            image.show()

            label = input("Enter label for this image (checked/not_checked): ")
            labeled_data.append((image_path, label))

            # Close the image
            image.close()

    return labeled_data

# Use this function to label your images
image_folder = 'path_to_your_image_folder'  # Replace with your image folder path
labeled_dataset = label_images(image_folder)

# Save the labeled data to a file for future use
with open('labeled_dataset.csv', 'w') as file:
    for path, label in labeled_dataset:
        file.write(f'{path},{label}\n')


In [ ]:
# Data generators for training and validation (for grayscale images)
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='label',
    class_mode='binary',
    target_size=(64, 64),
    batch_size=32,
    color_mode='grayscale'
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    y_col='label',
    class_mode='binary',
    target_size=(64, 64),
    batch_size=32,
    color_mode='grayscale'
)

# Define the model (for grayscale images)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(64, 64, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_generator, validation_data=test_generator, epochs=10)

# Save the model
model.save('checkbox_classifier.h5')
